# LIT Standalone Components

This notebook shows use of the [Language Interpretability Tool](https://pair-code.github.io/lit) components on a binary classifier for labelling statement sentiment (0 for negative, 1 for positive).

All LIT backend components (models, datasets, metrics, generators, etc.) are standalone Python classes, and can easily be used from Colab or another Python context without starting a server. This can be handy for development, of if you want to re-use components in an offline workflow.

Copyright 2021 Google LLC.
SPDX-License-Identifier: Apache-2.0

In [ ]:
# Install LIT and transformers packages. The transformers package is needed by the model and dataset we are using.
# Replace tensorflow-datasets with the nightly package to get up-to-date dataset paths.
!pip uninstall -y tensorflow-datasets
!pip install lit_nlp tfds-nightly transformers==4.1.1

## Load data

LIT's `Dataset` classes are just lists of records, plus spec information to describe each field.

In [1]:
import attr
import pandas as pd

from lit_nlp import notebook
from lit_nlp.examples.datasets import glue
from lit_nlp.examples.models import glue_models

sst_data = glue.SST2Data('validation')
sst_data.spec()

INFO:absl:Load dataset info from /root/tensorflow_datasets/glue/sst2/2.0.0
INFO:absl:Reusing dataset glue (/root/tensorflow_datasets/glue/sst2/2.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split validation, from /root/tensorflow_datasets/glue/sst2/2.0.0


{'label': CategoryLabel(required=True, vocab=['0', '1']),
 'sentence': TextSegment(required=True, default='')}

In [2]:
sst_data.examples[:10]

[{'label': '1', 'sentence': "it 's a charming and often affecting journey . "},
 {'label': '0', 'sentence': 'unflinchingly bleak and desperate '},
 {'label': '1',
  'sentence': 'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . '},
 {'label': '1',
  'sentence': "the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . "},
 {'label': '0', 'sentence': "it 's slow -- very , very slow . "},
 {'label': '1',
  'sentence': 'although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women . '},
 {'label': '0', 'sentence': 'a sometimes tedious film . '},
 {'label': '0',
  'sentence': "or doing last year 's taxes with your ex-wife . "},
 {'label': '1',
  'sentence': "you do n't have to know about music to appreciate the film 's easygoing blend of comedy and romance . "},
 {'label': '0',
  'sentence': "in exactly 89 minutes , most of whi

You can easily convert this to tabular form, too:

In [3]:
pd.DataFrame(sst_data.examples)

,sentence,label
0,it 's a charming and often affecting journey .,1
1,unflinchingly bleak and desperate,0
2,allows us to hope that nolan is poised to emba...,1
3,"the acting , costumes , music , cinematography...",1
4,"it 's slow -- very , very slow .",0
...,...,...
867,has all the depth of a wading pool .,0
868,a movie with a real anarchic flair .,1
869,a subject like this should inspire reaction in...,0
870,... is an arthritic attempt at directing by ca...,0


## Load a model and run inference

LIT's `Model` class defines a `predict()` function to perform inference. The `input_spec()` describes the expected inputs (it should be a subset of the dataset fields), and `output_spec()` describes the output.

In [4]:
# Fetch the trained model weights and load the model to analyze
!wget https://storage.googleapis.com/what-if-tool-resources/lit-models/sst2_tiny.tar.gz
!mkdir sst2_tiny
!tar -xvf sst2_tiny.tar.gz -C sst2_tiny

sentiment_model = glue_models.SST2Model('./sst2_tiny')
sentiment_model.input_spec(), sentiment_model.output_spec()

--2021-10-14 14:15:38--  https://storage.googleapis.com/what-if-tool-resources/lit-models/sst2_tiny.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16362834 (16M) [application/octet-stream]
Saving to: ‘sst2_tiny.tar.gz.1’

sst2_tiny.tar.gz.1  100%[===================>]  15.60M  81.4MB/s    in 0.2s    

2021-10-14 14:15:38 (81.4 MB/s) - ‘sst2_tiny.tar.gz.1’ saved [16362834/16362834]

./
./tokenizer_config.json
./tf_model.h5
./config.json
./train.history.json
./vocab.txt
./special_tokens_map.json


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


({'grad_class': CategoryLabel(required=False, vocab=['0', '1']),
  'input_embs_sentence': TokenEmbeddings(required=False, align='tokens'),
  'label': CategoryLabel(required=False, vocab=['0', '1']),
  'sentence': TextSegment(required=True, default=''),
  'tokens_sentence': Tokens(required=False, default=[], parent='sentence', mask_token=None)},
 {'cls_emb': Embeddings(required=True),
  'cls_grad': Gradients(required=True, grad_for='cls_emb', grad_target_field_key='grad_class'),
  'grad_class': CategoryLabel(required=False, vocab=['0', '1']),
  'input_embs_sentence': TokenEmbeddings(required=True, align='tokens_sentence'),
  'layer_0/attention': AttentionHeads(required=True, align_in='tokens', align_out='tokens'),
  'layer_1/attention': AttentionHeads(required=True, align_in='tokens', align_out='tokens'),
  'probas': MulticlassPreds(required=True, vocab=['0', '1'], null_idx=0, parent='label'),
  'token_grad_sentence': TokenGradients(required=True, align='tokens_sentence', grad_for='inpu

There's a lot of fields in the output spec, since this model returns embeddings, gradients, attention, and more. We can view it using Pandas to avoid too much clutter:

In [5]:
preds = list(sentiment_model.predict(sst_data.examples[:10]))
pd.DataFrame(preds)

,cls_emb,input_embs,layer_0/attention,layer_1/attention,probas,grad_class,tokens,tokens_sentence,input_embs_sentence,token_grad_sentence,cls_grad
0,"[0.004583381, -0.35583165, -1.0134914, 1.5898,...","[[0.017338583, -0.016962294, -0.501561, -0.010...","[[[0.31375167, 0.21949047, 0.57060295, 0.51083...","[[[0.08148516, 0.821547, 0.3571193, 0.4449412,...","[0.008859163, 0.9911409]",1,"[[CLS], it, ', s, a, charming, and, often, aff...","[it, ', s, a, charming, and, often, affecting,...","[[-0.02315473, 0.028583972, -0.043904603, -0.0...","[[-7.5466356e-05, -0.000115628325, 0.000267486...","[0.00018687929, 4.704453e-05, 4.0064948e-05, -..."
1,"[-0.53348106, -1.2959056, -2.0872705, -1.42752...","[[0.017338583, -0.016962294, -0.501561, -0.010...","[[[0.2999308, 0.019363625, 0.012921629, 0.0961...","[[[0.07893149, 0.9353795, 0.3541118, 0.8020245...","[0.8705278, 0.12947221]",0,"[[CLS], un, ##fl, ##in, ##ching, ##ly, bleak, ...","[un, ##fl, ##in, ##ching, ##ly, bleak, and, de...","[[-0.037998863, -0.024275642, 0.061621126, -0....","[[0.051445995, 0.044628385, -0.19452904, 0.001...","[-0.03781466, 0.00041138422, 0.002508022, 0.08..."
2,"[0.3407998, 0.20537995, -1.5944655, 1.2758317,...","[[0.017338583, -0.016962294, -0.501561, -0.010...","[[[0.14481069, 0.044249136, 0.13230227, 0.7652...","[[[0.02976306, 0.69763637, 0.8893215, 0.930776...","[0.011394653, 0.9886053]",1,"[[CLS], allows, us, to, hope, that, nolan, is,...","[allows, us, to, hope, that, nolan, is, poised...","[[-0.004202644, 0.065549225, -0.06381365, 0.01...","[[-0.0014428259, -0.00046781698, 0.0026154125,...","[0.00020122774, -0.0011052014, -0.00013908498,..."
3,"[0.9169466, 0.90732324, -2.097221, 0.38243675,...","[[0.017338583, -0.016962294, -0.501561, -0.010...","[[[0.1122977, 0.47727907, 0.08321068, 0.502560...","[[[0.028648494, 0.8274244, 0.6397511, 0.897796...","[0.032677714, 0.9673223]",1,"[[CLS], the, acting, ,, costumes, ,, music, ,,...","[the, acting, ,, costumes, ,, music, ,, cinema...","[[-0.043320682, 0.02815245, -0.027521769, -0.0...","[[-0.00048740115, -0.004126699, 0.006568273, 0...","[-0.012909396, -0.014266191, 0.00075534085, 0...."
4,"[-0.64637935, 0.06891413, -1.4176737, -1.76827...","[[0.017338583, -0.016962294, -0.501561, -0.010...","[[[0.30667877, 0.22150387, 0.55423075, 0.45006...","[[[0.034146205, 0.47710592, 0.15084067, 0.1739...","[0.97357225, 0.026427716]",0,"[[CLS], it, ', s, slow, -, -, very, ,, very, s...","[it, ', s, slow, -, -, very, ,, very, slow, .]","[[-0.02315473, 0.028583972, -0.043904603, -0.0...","[[0.00035544863, -0.00032099197, 0.00066477346...","[-0.0020571523, -0.0006151155, 0.0007737565, 0..."
5,"[0.27590856, -1.0817645, -0.84292084, 1.259297...","[[0.017338583, -0.016962294, -0.501561, -0.010...","[[[0.1573781, 0.21759886, 0.011105451, 0.49348...","[[[0.016249932, 0.8166377, 0.3254434, 0.557768...","[0.00933072, 0.99066925]",1,"[[CLS], although, laced, with, humor, and, a, ...","[although, laced, with, humor, and, a, few, fa...","[[0.03739236, 0.031962592, -0.045048732, -0.02...","[[-4.4891396e-05, -0.00037057052, 0.0008015726...","[-0.00010475758, -8.198748e-05, -2.4683137e-05..."
6,"[-0.7365836, -0.32791507, -1.1239178, -2.5176,...","[[0.017338583, -0.016962294, -0.501561, -0.010...","[[[0.3160024, 0.80639344, 0.27915937, 0.002850...","[[[0.062158547, 0.6758493, 0.49058425, 0.65609...","[0.92046547, 0.07953453]",0,"[[CLS], a, sometimes, ted, ##ious, film, ., [S...","[a, sometimes, ted, ##ious, film, .]","[[-0.04020428, -0.0076383823, -0.00210628, -0....","[[0.015292119, 0.0066954284, -0.027800344, 0.0...","[0.0010592741, 0.021598997, 0.0141387, 0.16900..."
7,"[0.5832945, 0.63631034, -2.8385923, -1.4457783...","[[0.017338583, -0.016962294, -0.501561, -0.010...","[[[0.25043583, 0.26420912, 0.12593256, 0.53413...","[[[0.06434139, 0.82461584, 0.6095685, 0.728478...","[0.81982076, 0.18017924]",0,"[[CLS], or, doing, last, year, ', s, taxes, wi...","[or, doing, last, year, ', s, taxes, with, you...","[[0.013928796, -0.019496849, 0.00

If we just want the predicted probabilites for each class, we can look at the `probas` field:

In [6]:
labels = sentiment_model.output_spec()['probas'].vocab
pd.DataFrame([p['probas'] for p in preds], columns=pd.Index(labels, name='label'))

label,0,1
0,0.008859,0.991141
1,0.870528,0.129472
2,0.011395,0.988605
3,0.032678,0.967322
4,0.973572,0.026428
5,0.009331,0.990669
6,0.920465,0.079535
7,0.819821,0.180179
8,0.012008,0.987992
9,0.970425,0.029575


## Salience methods

We can use different interpretability components as well. Here's an example running LIME to get a salience map. The output has entries for each input field, though here that's just one field named "sentence":

In [7]:
from lit_nlp.components import lime_explainer
lime = lime_explainer.LIME()

lime_results = lime.run(sst_data.examples[:1], sentiment_model, sst_data)[0]
lime_results

INFO:absl:Found text fields for LIME attribution: ['sentence']
INFO:absl:Explaining: it 's a charming and often affecting journey . 


{'sentence': TokenSalience(tokens=['it', "'s", 'a', 'charming', 'and', 'often', 'affecting', 'journey', '.'], salience=array([ 0.02434427,  0.04803366,  0.09268055,  0.31654744,  0.07632921,
         0.17353564,  0.14552917,  0.10201892, -0.02098114]))}

In [8]:
# Again, pretty-print output with Pandas. The SalienceMap object is just a dataclass defined using attr.s.
pd.DataFrame(attr.asdict(lime_results['sentence']))

,tokens,salience
0,it,0.024344
1,'s,0.048034
2,a,0.092681
3,charming,0.316547
4,and,0.076329
5,often,0.173536
6,affecting,0.145529
7,journey,0.102019
8,.,-0.020981


In [9]:
from lit_nlp.components import gradient_maps
ig = gradient_maps.IntegratedGradients()

ig_results = ig.run(sst_data.examples[:1], sentiment_model, sst_data)[0]
ig_results

INFO:absl:Found fields for integrated gradients: ['token_grad_sentence']


{'token_grad_sentence': TokenSalience(tokens=['it', "'", 's', 'a', 'charming', 'and', 'often', 'affecting', 'journey', '.'], salience=array([ 0.07291325, -0.0062626 ,  0.03211318,  0.06548805,  0.36459708,
         0.08916923,  0.10651349,  0.09290756,  0.14865497, -0.02138056],
       dtype=float32))}

In [10]:
# Again, pretty-print output with Pandas. The SalienceMap object is just a dataclass defined using attr.s.
pd.DataFrame(attr.asdict(ig_results['token_grad_sentence']))

,tokens,salience
0,it,0.072913
1,',-0.006263
2,s,0.032113
3,a,0.065488
4,charming,0.364597
5,and,0.089169
6,often,0.106513
7,affecting,0.092908
8,journey,0.148655
9,.,-0.021381


## Metrics

We can also compute metrics. The metrics components (via the `SimpleMetrics` API) will automatically detect compatible fields marked by the `parent` attribute - in this case, our model's `probas` field that should be scored against `label` in the input.

In [11]:
from lit_nlp.components import metrics
classification_metrics = metrics.MulticlassMetrics()
classification_metrics.run(sst_data.examples[:100], sentiment_model, sst_data)

[{'label_key': 'label',
  'metrics': {'accuracy': 0.83,
   'f1': 0.8349514563106797,
   'precision': 0.8431372549019608,
   'recall': 0.8269230769230769},
  'pred_key': 'probas'}]

## Generators

We can use counterfactual generators as well. Here's an example with a generator that simply scrambles words in a text segment.

In [12]:
from lit_nlp.components import scrambler
sc = scrambler.Scrambler()

sc_in = sst_data.examples[:5]
sc_out = sc.generate_all(sc_in, model=None, dataset=sst_data)
# The output is a list-of-lists, generated from each original example.
sc_out

[[{'label': '1',
   'sentence': "often affecting a it  journey and 's . charming"}],
 [{'label': '0', 'sentence': 'bleak unflinchingly  desperate and'}],
 [{'label': '1',
   'sentence': 'yet inventive that commercial to a  poised hope a us major allows . is embark career as nolan to filmmaker'}],
 [{'label': '1',
   'sentence': ", acting the production locales austere given music , all the  's and are costumes . cinematography sound , astounding"}],
 [{'label': '0', 'sentence': "-- , very  's . it slow very slow"}]]

In [14]:
# Format as a flat table for display, including original sentences
import itertools
for ex_in, exs_out in zip(sc_in, sc_out):
  for ex_out in exs_out:
    ex_out['original_sentence'] = ex_in['sentence']
pd.DataFrame(itertools.chain.from_iterable(sc_out), columns=['original_sentence', 'sentence', 'label'])

,original_sentence,sentence,label
0,it 's a charming and often affecting journey .,often affecting a it journey and 's . charming,1
1,unflinchingly bleak and desperate,bleak unflinchingly desperate and,0
2,allows us to hope that nolan is poised to emba...,yet inventive that commercial to a poised hop...,1
3,"the acting , costumes , music , cinematography...",", acting the production locales austere given ...",1
4,"it 's slow -- very , very slow .","-- , very 's . it slow very slow",0


# Running the LIT UI

Of course, you can always still use these components in the LIT UI, without leaving Colab.

In [ ]:
widget = notebook.LitWidget(models={'sentiment': sentiment_model}, 
                            datasets={'sst2': sst_data}, 
                            height=800)

In [ ]:
widget.render()